In [ ]:
#| default_exp handlers.maris_dump

# MARIS dump
> Data pipeline (handler) to convert global MARIS db dump into `NetCDF` format

The input data is a dump from already imported MARIS datasets.

**Dev. board**: https://trello.com/b/IszgV1bj/marisco

## Packages import

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from tqdm import tqdm
from pathlib import Path
import fastcore.all as fc
import pandas as pd

from marisco.callbacks import (Callback, Transformer, SanitizeLonLatCB, EncodeTimeCB)
from marisco.metadata import (GlobAttrsFeeder, BboxCB,
                              DepthRangeCB, TimeRangeCB,
                              ZoteroCB, KeyValuePairCB)
from marisco.configs import lut_path, cdl_cfg, cfg, nc_tpl_path, get_enum_dicts
from marisco.serializers import NetCDFEncoder

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
fname_in = Path().home() / 'pro/data/maris/MARIS_exportSample_20240312_tab2.txt'
dir_dest = '../../_data/output/dump'

## Utils

In [ ]:
# | export
def load_dump(fname): 
    return pd.read_csv(fname, sep='\t', encoding='ISO-8859-1')   

def load_data(df:pd.DataFrame, # MARIS global dump 
                 ref_id:int, # Reference id of interest
                 ):
    "Load specific MARIS dataset through its ref_id"
    lut = {
        'Sediment': 'sediment',
        'Seawater': 'seawater',
        'Suspended matter': 'suspended-matter',
        'Biota': 'biota'}
    dfs = {}
    for name, grp in df[df.ref_id  == ref_id].groupby('samptype'): 
        dfs[lut[name]] = grp
    return dfs

def get_zotero_key(dfs):
    return dfs[next(iter(dfs))][['zoterourl']].iloc[0].values[0].split('/')[-1]

def get_fname(dfs):
    id, name = dfs[next(iter(dfs))][['ref_id', 'displaytext']].iloc[0]
    name = name.replace(',', '').replace('.', '').replace('-', ' ').split(' ')
    return '-'.join(([str(id)] + name)) + '.nc'

## Load data

In [ ]:
df = load_dump(fname_in)

In [ ]:
df.head()

,ref_id,displaytext,samptype,nuclide_id,latitude,longitude,begperiod,endperiod,sampdepth,totdepth,...,bodypar_id,sedtype_id,volume,salinity,temperatur,sampmet_id,prepmet_id,counmet_id,activity,zoterourl
0,374,"Ostlund et al., 1987",Seawater,53,-53.0,103.016944,1978-02-18 00:00:00,NaN,1867.0,3808.0,...,0,0,NaN,34.731998,NaN,1,0,0,3.508917,https://www.zotero.org/groups/2432820/maris/it...
1,374,"Ostlund et al., 1987",Seawater,3,-53.0,103.016944,1978-02-18 00:00:00,NaN,1079.0,3808.0,...,0,0,NaN,34.740002,NaN,1,0,0,-144.700000,https://www.zotero.org/groups/2432820/maris/it...
2,374,"Ostlund et al., 1987",Seawater,53,-53.0,103.016944,1978-02-18 00:00:00,NaN,1079.0,3808.0,...,0,0,NaN,34.740002,NaN,1,0,0,3.320633,https://www.zotero.org/groups/2432820/maris/it...
3,374,"Ostlund et al., 1987",Seawater,1,-53.0,103.016944,1978-02-18 00:00:00,NaN,1140.0,3808.0,...,0,0,NaN,34.740002,NaN,12,0,0,-0.010000,https://www.zotero.org/groups/2432820/maris/it...
4,374,"Ostlund et al., 1987",Seawater,1,-53.0,103.016944,1978-02-18 00:00:00,NaN,1583.0,3808.0,...,0,0,NaN,34.743999,NaN,1,0,0,-0.050000,https://www.zotero.org/groups/2432820/maris/it...


In [ ]:
# 52, 191 (OSPAR)
ref_id = 174

In [ ]:
dfs = load_data(df, ref_id)

In [ ]:
# from marisco.inout import read_toml

# lut_src_dir_test = '../files/lut'
# cdl_enums_test = read_toml('../files/cdl.toml')['enums']

# enums = get_enum_dicts(lut_src_dir=lut_src_dir_test, 
#                        cdl_enums=cdl_enums_test)

# species = {v:k for k, v in enums['species_t'].items()}

# for idx in dfs['biota'].species_id.unique():
#     print(idx in species)

## Data transformation pipeline

### Normalize nuclide names

In [ ]:
#| export
def get_varnames_lut():
    fname = lut_path() / 'dbo_nuclide.xlsx'
    df_nuclide = pd.read_excel(fname, usecols=['nuclide_id', 'nc_name'])
    return df_nuclide.set_index('nuclide_id').to_dict()['nc_name']

In [ ]:
# | export
class RemapRdnNameCB(Callback):
    "Remap to MARIS radionuclide names."
    def __init__(self,
                 fn_lut=get_varnames_lut):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        for k in tfm.dfs.keys():
            tfm.dfs[k]['nuclide_id'] = tfm.dfs[k]['nuclide_id'].replace(lut)

In [ ]:
dfs = load_data(df, ref_id)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[RemapRdnNameCB()])

print(tfm()['sediment']['nuclide_id'].unique())

['ra228' 'k40' 'pu238' 'pu239' 'sr90' 'th232' 'u238' 'pb210' 'ra226']


### Rename columns

In [ ]:
dfs['sediment'].columns

Index(['ref_id', 'displaytext', 'samptype', 'nuclide_id', 'latitude',
       'longitude', 'begperiod', 'endperiod', 'sampdepth', 'totdepth',
       'uncertaint', 'unit_id', 'detection', 'area_id', 'species_id',
       'biogroup_id', 'bodypar_id', 'sedtype_id', 'volume', 'salinity',
       'temperatur', 'sampmet_id', 'prepmet_id', 'counmet_id', 'activity',
       'zoterourl'],
      dtype='object')

In [ ]:
#| export
# To be added: endperiod, totdepth, lab
def renaming_rules():
    vars = cdl_cfg()['vars']
    # Define column names renaming rules
    return {
        'latitude': vars['defaults']['lat']['name'],
        'longitude': vars['defaults']['lon']['name'],
        'begperiod': vars['defaults']['time']['name'],
        'sampdepth': vars['defaults']['depth']['name'],
        'uncertaint': vars['suffixes']['uncertainty']['name'],
        'unit_id': vars['suffixes']['unit']['name'],
        'detection': vars['suffixes']['detection_limit']['name'],
        'area_id': vars['defaults']['area']['name'], 
        'species_id': vars['bio']['species']['name'],
        'biogroup_id': vars['bio']['bio_group']['name'],
        'bodypar_id': vars['bio']['body_part']['name'],
        'sedtype_id': vars['sed']['sed_type']['name'],
        'volume': vars['suffixes']['volume']['name'],
        'salinity': vars['suffixes']['salinity']['name'],
        'temperatur': vars['suffixes']['temperature']['name'],
        'sampmet_id': vars['suffixes']['sampling_method']['name'],
        'prepmet_id': vars['suffixes']['preparation_method']['name'],
        'counmet_id': vars['suffixes']['counting_method']['name'],
        'activity': 'value',
        'nuclide_id': 'nuclide'
    }

In [ ]:
#| export
class RenameColumnCB(Callback):
    "Renaming variables to MARIS standard names."
    def __init__(self, renaming_rules=renaming_rules): fc.store_attr()
    def __call__(self, tfm):
        lut = renaming_rules()
        coi = lut.keys()
        for k in tfm.dfs.keys():
            # Select cols of interest
            tfm.dfs[k] = tfm.dfs[k].loc[:, coi]
            # Rename cols
            tfm.dfs[k].rename(columns=lut, inplace=True)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB()
    ])

print(tfm()['sediment'])

             lat        lon                 time  depth     unc  unit dl  \
14936 -62.166667 -94.276111  2003-01-31 00:00:00    NaN    5.00     4  =   
14937 -62.166667 -94.276111  2003-01-31 00:00:00    NaN  222.00     4  =   
14938 -62.166667 -94.276111  2003-01-31 00:00:00    NaN    0.10     4  =   
14939 -62.166667 -94.276111  2003-01-31 00:00:00    NaN    0.07     4  =   
14940 -62.166667 -94.276111  2003-01-31 00:00:00    NaN   40.00     4  =   
15080 -62.169444 -58.601944  2003-01-31 00:00:00    NaN    0.71     4  =   
15081 -62.169444 -58.601944  2003-01-31 00:00:00    NaN    0.79     4  =   
15082 -62.172500 -58.546667  2003-01-31 00:00:00    NaN    0.50     4  =   
15083 -62.172500 -58.546667  2003-01-31 00:00:00    NaN    0.43     4  =   
15084 -62.173889 -58.443889  2003-01-31 00:00:00    NaN    0.41     4  =   
15085 -62.173889 -58.443889  2003-01-31 00:00:00    NaN    0.47     4  =   
15086 -62.190556 -58.610556  2003-01-31 00:00:00    NaN    0.63     4  =   
15087 -62.19

### Drop NaN only columns

In [ ]:
#| export
class DropNAColumnsCB(Callback):
    "Drop variable containing only NaN or 'Not available' (id=0 in MARIS lookup tables)."
    def __init__(self, na_value=0):
        fc.store_attr()
        
    def isMarisNA(self, col): 
        return len(col.unique()) == 1 and col.iloc[0] == self.na_value
    
    def dropMarisNA(self, df):
        na_cols = [col for col in df.columns if self.isMarisNA(df[col])]
        return df.drop(labels=na_cols, axis=1)
        
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k] = tfm.dfs[k].dropna(axis=1, how='all')
            tfm.dfs[k] = self.dropMarisNA(tfm.dfs[k])

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB()
    ])

print(tfm()['sediment'])

             lat        lon                 time     unc  unit dl  area  \
14936 -62.166667 -94.276111  2003-01-31 00:00:00    5.00     4  =  1907   
14937 -62.166667 -94.276111  2003-01-31 00:00:00  222.00     4  =  1907   
14938 -62.166667 -94.276111  2003-01-31 00:00:00    0.10     4  =  1907   
14939 -62.166667 -94.276111  2003-01-31 00:00:00    0.07     4  =  1907   
14940 -62.166667 -94.276111  2003-01-31 00:00:00   40.00     4  =  1907   
15080 -62.169444 -58.601944  2003-01-31 00:00:00    0.71     4  =  1907   
15081 -62.169444 -58.601944  2003-01-31 00:00:00    0.79     4  =  1907   
15082 -62.172500 -58.546667  2003-01-31 00:00:00    0.50     4  =  1907   
15083 -62.172500 -58.546667  2003-01-31 00:00:00    0.43     4  =  1907   
15084 -62.173889 -58.443889  2003-01-31 00:00:00    0.41     4  =  1907   
15085 -62.173889 -58.443889  2003-01-31 00:00:00    0.47     4  =  1907   
15086 -62.190556 -58.610556  2003-01-31 00:00:00    0.63     4  =  1907   
15087 -62.190556 -58.6105

### Sanitize detection limit values

In [ ]:
#| export
def get_dl_lut():
    fname = lut_path() / 'dbo_detectlimit.xlsx'
    df_nuclide = pd.read_excel(fname, usecols=['name', 'id'])
    return df_nuclide.set_index('name').to_dict()['id']

In [ ]:
get_dl_lut()

{'Not Available': 0, '=': 1, '<': 2, 'ND': 3, 'DE': 4}

In [ ]:
#| export
class SanitizeDetectionLimitCB(Callback):
    "Assign Detection Limit name to its id based on MARIS nomenclature."
    def __init__(self,
                 fn_lut=get_dl_lut):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        for k in tfm.dfs.keys():
            tfm.dfs[k]['dl'] = tfm.dfs[k]['dl'].replace(lut)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB()
    ])

print(tfm()['sediment'])

             lat        lon                 time     unc  unit  dl  area  \
14936 -62.166667 -94.276111  2003-01-31 00:00:00    5.00     4   1  1907   
14937 -62.166667 -94.276111  2003-01-31 00:00:00  222.00     4   1  1907   
14938 -62.166667 -94.276111  2003-01-31 00:00:00    0.10     4   1  1907   
14939 -62.166667 -94.276111  2003-01-31 00:00:00    0.07     4   1  1907   
14940 -62.166667 -94.276111  2003-01-31 00:00:00   40.00     4   1  1907   
15080 -62.169444 -58.601944  2003-01-31 00:00:00    0.71     4   1  1907   
15081 -62.169444 -58.601944  2003-01-31 00:00:00    0.79     4   1  1907   
15082 -62.172500 -58.546667  2003-01-31 00:00:00    0.50     4   1  1907   
15083 -62.172500 -58.546667  2003-01-31 00:00:00    0.43     4   1  1907   
15084 -62.173889 -58.443889  2003-01-31 00:00:00    0.41     4   1  1907   
15085 -62.173889 -58.443889  2003-01-31 00:00:00    0.47     4   1  1907   
15086 -62.190556 -58.610556  2003-01-31 00:00:00    0.63     4   1  1907   
15087 -62.19

### Parse time

In [ ]:
#| export
class ParseTimeCB(Callback):
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k]['time'] = pd.to_datetime(tfm.dfs[k].time, format='ISO8601')

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ParseTimeCB()
    ])

print(tfm()['sediment'])

             lat        lon       time     unc  unit  dl  area  sed_type  \
14936 -62.166667 -94.276111 2003-01-31    5.00     4   1  1907         0   
14937 -62.166667 -94.276111 2003-01-31  222.00     4   1  1907         0   
14938 -62.166667 -94.276111 2003-01-31    0.10     4   1  1907         0   
14939 -62.166667 -94.276111 2003-01-31    0.07     4   1  1907         0   
14940 -62.166667 -94.276111 2003-01-31   40.00     4   1  1907         0   
15080 -62.169444 -58.601944 2003-01-31    0.71     4   1  1907        12   
15081 -62.169444 -58.601944 2003-01-31    0.79     4   1  1907        12   
15082 -62.172500 -58.546667 2003-01-31    0.50     4   1  1907         6   
15083 -62.172500 -58.546667 2003-01-31    0.43     4   1  1907         6   
15084 -62.173889 -58.443889 2003-01-31    0.41     4   1  1907         0   
15085 -62.173889 -58.443889 2003-01-31    0.47     4   1  1907         0   
15086 -62.190556 -58.610556 2003-01-31    0.63     4   1  1907         2   
15087 -62.19

### Reshape: long to wide

In [ ]:
#| export
class ReshapeLongToWide(Callback):
    "Convert data from long to wide with renamed columns."
    def __init__(self, value_col='nuclide'):
        fc.store_attr()
        self.derived_cols = [value['name'] for value in cdl_cfg()['vars']['suffixes'].values()]
    
    def renamed_cols(self, cols):
        return [f'{inner}_{outer}' if inner else outer for outer, inner in cols]

    def pivot(self, df):
        derived_coi = [col for col in self.derived_cols if col in df.columns]
        df.reset_index(names='sample', inplace=True)
        idx = list(set(df.columns) - set([self.value_col] + derived_coi))
        return df.pivot_table(index=idx,
                              columns=self.value_col,
                              values=derived_coi,
                              fill_value=0,
                              aggfunc=lambda x: x
                              ).reset_index()

    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k] = self.pivot(tfm.dfs[k])
            tfm.dfs[k].columns = self.renamed_cols(tfm.dfs[k].columns)
            # tfm.dfs[k].index.name = 'sample'
            tfm.dfs[k].set_index('sample', inplace=True)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ParseTimeCB(),
    ReshapeLongToWide()
    ])

print(tfm()['sediment'])

        sed_type   value       time        lon  area        lat  k40_counmet  \
sample                                                                         
14939          0    0.19 2003-01-31 -94.276111  1907 -62.166667            0   
14938          0    0.30 2003-01-31 -94.276111  1907 -62.166667            0   
15084          0   14.26 2003-01-31 -58.443889  1907 -62.173889            0   
15104          0   15.07 2003-01-31 -58.490833  1907 -62.157500            0   
15107          0   15.07 2003-01-31 -58.368056  1907 -62.160556            0   
15085          0   16.12 2003-01-31 -58.443889  1907 -62.173889            0   
15108          0   16.12 2003-01-31 -58.368056  1907 -62.160556            0   
14936          0   17.00 2003-01-31 -94.276111  1907 -62.166667            0   
15110          0   21.00 2003-01-31 -94.276111  1907 -62.166667            0   
14940          0   31.00 2003-01-31 -94.276111  1907 -62.166667            0   
15109          0  105.00 2003-01-31 -94.

### Encode time (seconds since ...)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ParseTimeCB(),
    ReshapeLongToWide(),
    EncodeTimeCB(cfg())
    ])

print(tfm()['sediment'])

        sed_type   value        time        lon  area        lat  k40_counmet  \
sample                                                                          
14939          0    0.19  1043971200 -94.276111  1907 -62.166667            0   
14938          0    0.30  1043971200 -94.276111  1907 -62.166667            0   
15084          0   14.26  1043971200 -58.443889  1907 -62.173889            0   
15104          0   15.07  1043971200 -58.490833  1907 -62.157500            0   
15107          0   15.07  1043971200 -58.368056  1907 -62.160556            0   
15085          0   16.12  1043971200 -58.443889  1907 -62.173889            0   
15108          0   16.12  1043971200 -58.368056  1907 -62.160556            0   
14936          0   17.00  1043971200 -94.276111  1907 -62.166667            0   
15110          0   21.00  1043971200 -94.276111  1907 -62.166667            0   
14940          0   31.00  1043971200 -94.276111  1907 -62.166667            0   
15109          0  105.00  10

### Sanitize coordinates

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ParseTimeCB(),
    ReshapeLongToWide(),
    EncodeTimeCB(cfg()),
    SanitizeLonLatCB()
    ])

# print(tfm()['sediment'])
df_debug = tfm()['sediment']

## Encode to NetCDF

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ParseTimeCB(),
    ReshapeLongToWide(),
    EncodeTimeCB(cfg()),
    SanitizeLonLatCB()
    ])

dfs_tfm = tfm()

In [ ]:
tfm.logs

['Remap to MARIS radionuclide names.',
 'Renaming variables to MARIS standard names.',
 "Drop variable containing only NaN or 'Not available' (id=0 in MARIS lookup tables).",
 'Assign Detection Limit name to its id based on MARIS nomenclature.',
 'Convert data from long to wide with renamed columns.',
 'Encode time as `int` representing seconds since xxx',
 'Drop row when both longitude & latitude equal 0.']

In [ ]:
#| export
kw = ['oceanography', 'Earth Science > Oceans > Ocean Chemistry> Radionuclides',
      'Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure',
      'Earth Science > Oceans > Ocean Chemistry > Ocean Tracers, Earth Science > Oceans > Marine Sediments',
      'Earth Science > Oceans > Ocean Chemistry, Earth Science > Oceans > Sea Ice > Isotopes',
      'Earth Science > Oceans > Water Quality > Ocean Contaminants',
      'Earth Science > Biological Classification > Animals/Vertebrates > Fish',
      'Earth Science > Biosphere > Ecosystems > Marine Ecosystems',
      'Earth Science > Biological Classification > Animals/Invertebrates > Mollusks',
      'Earth Science > Biological Classification > Animals/Invertebrates > Arthropods > Crustaceans',
      'Earth Science > Biological Classification > Plants > Macroalgae (Seaweeds)']

In [ ]:
#| export
def get_attrs(tfm, zotero_key='26VMZZ2Q', kw=kw):
    return GlobAttrsFeeder(tfm.dfs, cbs=[
        BboxCB(),
        DepthRangeCB(),
        TimeRangeCB(cfg()),
        ZoteroCB(zotero_key, cfg=cfg()),
        KeyValuePairCB('keywords', ', '.join(kw)),
        KeyValuePairCB('publisher_postprocess_logs', ', '.join(tfm.logs))
        ])()

In [ ]:
get_attrs(tfm, zotero_key='3W354SQG', kw=kw)

{'geospatial_lat_min': '-58.331388888888895',
 'geospatial_lat_max': '-62.084722222222226',
 'geospatial_lon_min': '-94.2761111111111',
 'geospatial_lon_max': '-62.2225',
 'geospatial_bounds': 'POLYGON ((-94.2761111111111 -62.2225, -58.331388888888895 -62.2225, -58.331388888888895 -62.084722222222226, -94.2761111111111 -62.084722222222226, -94.2761111111111 -62.2225))',
 'time_coverage_start': '2003-01-31T00:00:00',
 'time_coverage_end': '2003-01-31T00:00:00',
 'title': 'Radioactivity Monitoring of the Irish Marine Environment 1991 and 1992',
 'summary': '',
 'creator_name': '[{"creatorType": "author", "firstName": "A.", "lastName": "McGarry"}, {"creatorType": "author", "firstName": "S.", "lastName": "Lyons"}, {"creatorType": "author", "firstName": "C.", "lastName": "McEnri"}, {"creatorType": "author", "firstName": "T.", "lastName": "Ryan"}, {"creatorType": "author", "firstName": "M.", "lastName": "O\'Colmain"}, {"creatorType": "author", "firstName": "J.D.", "lastName": "Cunningham"}]'

In [ ]:
#| export
def encode(fname_in, fname_out, nc_tpl_path, **kwargs):
    df = load_dump(fname_in)
    ref_ids = kwargs.get('ref_ids', df.ref_id.unique())
    print('Encoding ...')
    for ref_id in tqdm(ref_ids, leave=False):
        dfs = load_data(df, ref_id)
        print(get_fname(dfs))
        tfm = Transformer(dfs, cbs=[
            RemapRdnNameCB(),
            RenameColumnCB(),
            DropNAColumnsCB(),
            SanitizeDetectionLimitCB(),
            ParseTimeCB(),
            ReshapeLongToWide(),
            EncodeTimeCB(cfg()),
            SanitizeLonLatCB()
            ])
        
        # species_lut = get_maris_species(fname_in, 'species_helcom.pkl')
        # enums_xtra = {
        #     'species_t': {info['name']: info['id'] 
        #                   for info in species_lut.values() if info['name'] != ''}
        # }
        encoder = NetCDFEncoder(tfm(), 
                                src_fname=nc_tpl_path,
                                dest_fname=Path(fname_out) / get_fname(dfs), 
                                global_attrs=get_attrs(tfm, zotero_key=get_zotero_key(dfs), kw=kw),
                                verbose=kwargs.get('verbose', False)
                                # enums_xtra=enums_xtra
                                )
        encoder.encode()

### Single dataset

In [ ]:
nc_tpl_path()

Path('/Users/franckalbinet/.marisco/maris-template.nc')

In [ ]:
ref_id = 174
encode(fname_in, dir_dest, nc_tpl_path(), verbose=True, ref_ids=[ref_id])

Encoding ...


  0%|          | 0/1 [00:00<?, ?it/s]

174-Hurtado-Bermúdez-et-al-2018.nc


Group: biota, Variable: sample
Group: biota, Variable: lon
Group: biota, Variable: lat
Group: biota, Variable: depth
Group: biota, Variable: time
Group: biota, Variable: area
Group: biota, Variable: bio_group
Group: biota, Variable: species


ValueError: trying to assign illegal value to Enum variable

### All datasets

In [ ]:
encode(fname_in, dir_dest, nc_tpl_path(), verbose=False)

Encoding ...


  0%|          | 0/462 [00:00<?, ?it/s]

374-Ostlund-et-al-1987.nc


  0%|          | 1/462 [00:04<31:05,  4.05s/it]

401-Olsen-et-al-2016.nc


...

 37%|███▋      | 171/462 [06:02<08:54,  1.84s/it]

168-Vivo-Vilches-et-al-2018.nc


 37%|███▋      | 172/462 [06:04<08:42,  1.80s/it]

174-Hurtado-Bermúdez-et-al-2018.nc


ValueError: trying to assign illegal value to Enum variable